# Readme
This is a demo notebook that shows how a personal schema is loaded and how to add data to it.
It's also great to test how to add new tables and how this affects your pipeline.

Once you are done with the tutorial, drop your schema by calling: `schema.drop()`.
This is also described at the very end of the tutorial.

If you have not installed `djimaging`, please go to project directory and install it via `pip install -e .`, see also below.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import datajoint as dj

# Setup

In [ ]:
username = !whoami
username = username[0]
username

In [ ]:
home_directory = os.path.expanduser("~")
home_directory

In [ ]:
# Path to djimaging
path_to_djimaging = f'{home_directory}/github/eulerlab/'

# Clone djimaging if you haven't downloaded it yet
# assert os.isdir(path_to_djimaging), 'Create target folder before cloning djimaging'
# !cd {path_to_djimaging} && git clone git@github.com:eulerlab/djimaging.git

# Install djimaging if not done yet
# assert os.isdir(os.path.join(path_to_djimaging, 'djimaging')), 'Create target folder before cloning djimaging'
# !cd {path_to_djimaging}/djimaging/ && sudo pip install -e .

# Data Base

## Prepare dj config

In [ ]:
# Set config file
config_file = f'{home_directory}/datajoint/dj_{username}_conf.json'
assert os.path.isfile(config_file), f'Set the path to your config file: {config_file}'

In [ ]:
# Define a schema name or use the default name for your personal test schema
# It should start with ageuler and have some meaningful name after that
schema_name = f"ageuler_{username}_test"

In [ ]:
# Do you want to use the RGC classifier?
# If so, make sure to add the respective tables into your schema.
use_rgc_classifier = True

if use_rgc_classifier:  # Define any existing outputfolder for the classifier to be saved
    output_folder = f'{home_directory}/datajoint/rgc_classifier'
    assert os.path.isdir(output_folder), f'Set path to output directory: {output_folder}'

In [ ]:
# Load configuration for user
dj.config.load(config_file)
dj.config['schema_name'] = schema_name

if use_rgc_classifier:
    from djimaging.tables.classifier.rgc_classifier import prepare_dj_config_rgc_classifier
    prepare_dj_config_rgc_classifier(output_folder)

print("schema_name:", dj.config['schema_name'])
dj.conn()

## Create or load schema

In [ ]:
# Import your own schema, which defines the tables you will have in your database and how they are connected.
from djimaging.schemas.tutorial_schema import *

In [ ]:
if use_rgc_classifier:
    try:
        CelltypeAssignment()
    except Exception as e:
        import warnings
        warnings.warn("""
            If you want to include the RGC classifier in your database, you have to copy the respective tables from 
            djimaging/schemas/full_rgc_schema.py 
            to the schema you just imported (see above).
            Then restart the notebook and try again.
            """)

## Important note

If the schema with the name `schema_name = f"ageuler_{username}_test"` already exists, it is important that the schema definition here is the same as it was when the schema was created.
If you did the other tutorial first, and did not delete (=drop) the schema afterwards, this will not be the case, for example.
Then you already have a schema with the same name but different tables, the first being based on the schema `rgc_classifier_schema`, and this one being based on `my_schema`. This can result in a variety of problems, so you either have to change the schema name here or drop the old schema first.

Outside of this tutorial, in most cases, you want exactly one schema per project to never run into this problem.

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, create_schema=True, create_tables=True)
schema

# ERD

In [ ]:
# Draw the ERD of your schema, this may take a while
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# 
# dj.ERD(schema)

## Upload user

In [ ]:
userinfo = {
    'experimenter': 'DataJointTestData', # Replace this if you want to use your own data
    'data_dir': '/gpfs01/euler/data/Data/DataJointTestData/xy-RGCs/', # Replace this if you want to use your own data
    'datatype_loc': 0,
    'animal_loc': 1,
    'region_loc': 2,
    'field_loc': 3,
    'stimulus_loc': 4,
    'cond1_loc': 5,
    'cond2_loc': 6,
    'cond3_loc': 7,
}

assert os.path.isdir(userinfo['data_dir'])

In [ ]:
UserInfo().upload_user(userinfo)
UserInfo()

In [ ]:
# Plot the data files in the selected folder
UserInfo().plot1(key=None, show_pre=False, show_raw=False, show_header=True)

In [ ]:
RawDataParams().add_default()
RawDataParams()

## Populate data

### Experiments

In [ ]:
Experiment().rescan_filesystem(verboselvl=2)
Experiment()

### Fields

In [ ]:
Field().rescan_filesystem(verboselvl=2)

In [ ]:
Field()

In [ ]:
# If you call plot1 on a djimaging table, it will plot the given key.
# If you pass key=None or no key it will pick a key at random.
# This is implemented for most core tables and can be useful to get a quick impression of the data.
Field().plot1(key=None)

### Stimuli

#### Add default stimuli

In [ ]:
# To compute receptive fields, the noise stimulus trace must be loaded and set!
import h5py

with h5py.File("/gpfs01/euler/data/Resources/Stimulus/noise.h5", "r") as f:
    noise_stimulus = f['stimulusarray'][:].T.astype(int)

In [ ]:
Stimulus().add_nostim(skip_duplicates=True)
Stimulus().add_chirp(spatialextent=1000, stim_name='gChirp', alias="chirp_gchirp_globalchirp", skip_duplicates=True)
Stimulus().add_chirp(spatialextent=300, stim_name='lChirp', alias="lchirp_localchirp", skip_duplicates=True)
Stimulus().add_noise(stim_name='noise', pix_n_x=20, pix_n_y=15, pix_scale_x_um=30, pix_scale_y_um=30, stim_trace=noise_stimulus, skip_duplicates=True)
Stimulus().add_movingbar(skip_duplicates=True)

In [ ]:
# Add custom stimulus if needed
# Stimulus().add_stimulus(stim_name='my_stimulus', alias="stimname_stimnameabbr_alternativename", isrepeated=True, ntrigger_rep=6,
#                         trial_info=[1, 2, 3, 4, 5, 6], skip_duplicates=True)

In [ ]:
Stimulus()

#### Presentations

In [ ]:
Presentation().populate(processes=20, display_progress=True)

In [ ]:
Presentation()

In [ ]:
Presentation().plot1(key=None)

# AutoROIs

Skip/delete this if you don't want to use AutoROIs.

In [ ]:
# If you have save some of you AutoROIs ROI masks you can load them here.
RoiMask().rescan_filesystem(verboselvl=2)
RoiMask()

In [ ]:
# Find all the fields that still require a ROI mask.
missing_fields = RoiMask().list_missing_field()

In [ ]:
field_key = missing_fields.pop()  # Pick one field

# Load ROI canvas, draw the ROI mask, clean it if you want, shift if you want.
# You can then save it to a file to be able to load it again later.
roi_canvas = RoiMask().draw_roi_mask(field_key=field_key, canvas_width=30)
roi_canvas.start_gui()

In [ ]:
# Load the just saved ROI mask
RoiMask().rescan_filesystem(verboselvl=2)

In [ ]:
RoiMask().plot1()

# Rois

In [ ]:
Roi().populate(processes=20, display_progress=True)
Roi()

In [ ]:
Roi().plot1(key=None)

### Traces

In [ ]:
Traces().populate(processes=20, display_progress=True)
Traces()

In [ ]:
PreprocessParams().add_default(skip_duplicates=True)
PreprocessParams()

In [ ]:
PreprocessTraces().populate(processes=20, display_progress=True)
PreprocessTraces()

In [ ]:
PreprocessTraces().plot1(key=None)

In [ ]:
Snippets().populate(processes=20, display_progress=True)
Snippets()

In [ ]:
Snippets().plot1(key=None)

In [ ]:
Averages().populate(processes=20, display_progress=True)
Averages()

In [ ]:
Averages().plot({"stim_name": "gChirp"})

## Quality and statistics

In [ ]:
ChirpQI().populate(display_progress=True, processes=20)
ChirpQI()

In [ ]:
OsDsIndexes().populate(display_progress=True, processes=20)
OsDsIndexes()

## Recording location

In [ ]:
OpticDisk().populate(processes=20, display_progress=True)
OpticDisk()

In [ ]:
RelativeFieldLocation().populate(processes=20, display_progress=True)
RelativeFieldLocation()

In [ ]:
RelativeFieldLocation().plot()

In [ ]:
RetinalFieldLocation().populate(processes=20, display_progress=True)
RetinalFieldLocation()

In [ ]:
RetinalFieldLocation().plot()

# RGC Classifier

Skip/delete this is you don't want to use the RGC classifier

In [ ]:
try:
    CelltypeAssignment()
except Exception as e:
    import warnings
    warnings.warn("""
        If you want to include the RGC classifier in your database, you have to copy the respective tables from 
        djimaging/schemas/rgc_classifier_schema.py 
        to the schema you just imported (see above).
        Then restart the notebook and try again.
        """)

### Add classifier method

In [ ]:
ClassifierMethod().add_default(skip_duplicates=True)
ClassifierMethod()

## Add training data

In [ ]:
ClassifierTrainingData().add_default(skip_duplicates=True)
ClassifierTrainingData()

## Train classifier

In [ ]:
Classifier().populate()
Classifier()

## Assign RGC types

In [ ]:
Baden16Traces().populate(display_progress=True, processes=20)

In [ ]:
CelltypeAssignment().populate(display_progress=True)
CelltypeAssignment()

In [ ]:
CelltypeAssignment().plot(threshold_confidence=0.0)
CelltypeAssignment().plot(threshold_confidence=0.25)
CelltypeAssignment().plot(threshold_confidence=0.5)

# Clean up

If you are done with the tutorial you can delete (=drop) your schema again and create a schema with a more meaningful name than `ageuler_{username}_test`.

In [ ]:
if input("Continue with cleaning up? (yes/no))") != "yes":
    raise ValueError('Enter yes if you wish to continue.')

schema.drop()